In [1]:
import pandas as pd
import numpy as np

In [2]:
!ls ..

code_file            cz_sh_train_0816.csv
cz_sh_test_0816.csv  data_clean_0811.csv


In [3]:
data = pd.read_csv('../data_clean_0811.csv',index_col=0)
data = data[data['rtc']>=1627747200000] # 8.1日之后数据
data.head()

,robot_id,rtc,protective_stop,task_state,rapid_rate,sum_running_time,ins_average_power,sum_running_circles,joint_actual_position,current_fluctuation,inst_temperature,inst_voltage,joint,health
23212,Zu7250098,1628643756145,0,4,1.0,12589525,15.000000,3.016309e+05,51.366797,1.0,54.0,47.0,1,1
23216,Zu3100161,1628643755723,0,4,1.0,12364110,3.400000,1.070656e+06,179.126535,1.5,46.0,48.0,1,1
23222,Zu3100161,1628643752846,0,4,1.0,12364106,3.800000,1.070655e+06,249.018715,1.1,46.0,48.0,1,1
23224,Zu7250098,1628643752716,0,4,1.0,12589521,36.400002,3.016302e+05,56.552356,0.8,54.0,48.0,1,1
23231,Zu3100161,1628643749708,0,4,1.0,12364104,12.400000,1.070654e+06,134.857509,2.2,46.0,48.0,1,1


In [4]:
data.shape

(1152912, 14)

选取二关节数据

In [5]:
joint_2 = data[data['joint']==2]

In [6]:
print("health=1:",joint_2[joint_2['health']==1].shape)
print("health=0:",joint_2[joint_2['health']==0].shape)

health=1: (176174, 14)
health=0: (15978, 14)


二关节坏的机器人

In [7]:
joint_2.loc[joint_2['health'] ==0,'robot_id'].unique()

array(['Zu3100002', 'Zu7250085'], dtype=object)

### Zu7250085作为测试集

In [10]:
joint_2.loc[joint_2['robot_id']=='Zu3100002'].shape

(15620, 14)

In [11]:
joint_2.loc[joint_2['robot_id']=='Zu7250085'].shape

(358, 14)

取出好坏关节

In [14]:
joint_2_great = joint_2.loc[joint_2['health']==1]

In [15]:
joint_2_broken = joint_2.loc[joint_2['health'] == 0]

In [17]:
joint_2_broken[joint_2_broken['robot_id']== 'Zu3100002'].shape

(15620, 14)

In [18]:
test = joint_2_broken[joint_2_broken['robot_id']== 'Zu7250085']

In [20]:
joint_2_broken.shape

(15978, 14)

In [21]:
joint_2_great.shape

(176174, 14)

从好的二关节里面取出16000放入训练集，400放入测试集

In [22]:
joint_2_temp = joint_2_great.sample(n=16400)

In [24]:
test = pd.concat([joint_2_temp.iloc[:400,:],test])

In [32]:
train = joint_2_broken[joint_2_broken['robot_id']=='Zu3100002']

In [33]:
train = pd.concat([joint_2_temp.iloc[400:,:],train])

In [34]:
from sklearn.utils import shuffle

In [35]:
train = shuffle(train)

In [36]:
train.loc[train['health']==0,'robot_id'].unique()

array(['Zu3100002'], dtype=object)

In [37]:
train.to_csv('../cz_sh_train_0816.csv')

In [38]:
test = shuffle(test)
test

,robot_id,rtc,protective_stop,task_state,rapid_rate,sum_running_time,ins_average_power,sum_running_circles,joint_actual_position,current_fluctuation,inst_temperature,inst_voltage,joint,health
1511560,Zu7250085,1627886253339,0,4,0.64,4576548,52.700001,163913.593750,118.790608,1.7,39.0,48.0,2,0
40446,Zu3100161,1628637899033,0,4,1.00,12358252,0.000000,105882.398438,87.266460,0.5,49.0,48.0,2,1
1647194,Zu7250396,1627812418756,0,4,1.00,4444907,0.000000,55359.199219,90.004610,0.1,44.0,48.0,2,1
1531875,Zu7250085,1627875274882,0,4,0.29,4571132,19.600000,163863.093750,80.999238,0.6,39.0,48.0,2,0
963563,Zu7250397,1628163987584,0,4,0.50,1153533,0.000000,50574.199219,62.735370,0.2,44.0,47.0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1511926,Zu7250085,1627886059363,0,4,0.64,4576355,7.200000,163892.500000,130.234884,1.0,39.0,47.0,2,0
961070,Zu7250396,1628165628920,0,4,1.00,4525003,0.000000,55370.898438,90.004610,0.1,36.0,48.0,2,1
1531862,Zu7250085,1627875281290,0,4,0.29,4571138,0.000000,163863.203125,79.927111,0.1,39.0,48.0,2,0
1716164,Zu7250396,1627774528265,0,4,1.00,4407008,0.000000,55359.199219,90.004610,0.1,42.0,48.0,2,1


In [39]:
test.to_csv('../cz_sh_test_0816.csv')

In [40]:
test.loc[test['health']==0,'robot_id'].unique()

array(['Zu7250085'], dtype=object)